<a href="https://colab.research.google.com/github/hvhq/classificationp1/blob/master/apply_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os.path
if not os.path.exists('gdrive'):
  drive.mount('gdrive')

In [2]:
import pandas as pd
from google.colab import files
import sklearn
from sklearn import preprocessing
import numpy as np
import math
from sklearn.metrics import mean_squared_error
import json

In [3]:
!pip install ipython-autotime
%load_ext autotime
!pip install kaggle
!mkdir -p /root/.kaggle

#upload kaggle token file: kaggle.json
#files.upload()
kaggletoken={"username":"huynhvanhien","key":"82d7e5f0d5f149aedf8f040ba7c74c2a"}
with open('/root/.kaggle/kaggle.json', 'w')as kaggletokenfile:
  json.dump(kaggletoken, kaggletokenfile)
#move to /.kaggle

#!cp kaggle.json /root/.kaggle/
!chmod 600 /.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d nehalbirla/vehicle-dataset-from-cardekho

  0% 0.00/59.6k [00:00<?, ?B/s]
100% 59.6k/59.6k [00:00<00:00, 21.5MB/s]
time: 720 ms


In [5]:
import zipfile
zipreference=zipfile.ZipFile('vehicle-dataset-from-cardekho.zip', 'r')
zipreference.extractall('Data')
zipreference.close()


time: 10.3 ms


In [6]:
data=pd.read_csv("Data/car data.csv")
if 'Seller_Type' in data.columns:
  data=data.drop(['Seller_Type'], axis=1)
if 'Car_Name' in data.columns:
  data=data.drop(['Car_Name', 'Owner'], axis=1)

#filter one species
#datab=pd.DataFrame(columns=['Species','Weight','Length1','Length2', 'Length3', 'Height', 'Width'])
#for i in range(0, lendata):
#  if data.Species[i]=='Bream':
#    datab=datab.append(data.iloc[i], ignore_index=True)
#data=datab
#lendata=len(data)

#encoder string to numeric
encoder=sklearn.preprocessing.LabelEncoder()
encoder.fit(data.Fuel_Type)
data.Fuel_Type=encoder.transform(data.Fuel_Type)
encoder.fit(data.Transmission)
data.Transmission=encoder.transform(data.Transmission)

lendata=len(data)
#change domain to [0,1]
for i in range(0, len(data.columns)):
  #print(data.iloc[:,i])
  data.iloc[:,i]=(data.iloc[:,i]-([min(data.iloc[:,i])]*lendata))/([max(data.iloc[:,i])-min(data.iloc[:,i])]*lendata)
print(data)

         Year  Selling_Price  ...  Fuel_Type  Transmission
0    0.733333       0.093123  ...        1.0           1.0
1    0.666667       0.133238  ...        0.5           1.0
2    0.933333       0.204871  ...        1.0           1.0
3    0.533333       0.078797  ...        1.0           1.0
4    0.733333       0.128940  ...        0.5           1.0
..        ...            ...  ...        ...           ...
296  0.866667       0.269341  ...        0.5           1.0
297  0.800000       0.111748  ...        1.0           1.0
298  0.400000       0.093123  ...        1.0           1.0
299  0.933333       0.326648  ...        0.5           1.0
300  0.866667       0.148997  ...        1.0           1.0

[301 rows x 6 columns]
time: 67.7 ms


In [7]:

#split to x, y train and test
x=data.drop('Selling_Price', axis=1)
y=pd.DataFrame(data.Selling_Price)
trainOTestRate=3
xTrain=x
xTest=pd.DataFrame(columns=x.columns)
yTrain=y
yTest=pd.DataFrame(columns=['Selling_Price'])
for i in range(0, len(data)):
  if i%(trainOTestRate+1)==0:
    xTest=xTest.append(x.iloc[i], ignore_index=True)
    xTrain=xTrain.drop([i])
    yTest=yTest.append(y.iloc[i], ignore_index=True)
    yTrain=yTrain.drop([i])
xTrain=xTrain.reset_index(drop=True)
yTrain=yTrain.reset_index(drop=True)

#xTrain=xTrain.to_numpy()
#xTest=xTest.to_numpy()
#yTrain=yTrain.to_numpy()
#yTest=yTest.to_numpy()
print("train")
print(xTrain)
print(yTrain)
print("test")
print(xTest)
print(yTest)


train
         Year  Present_Price  Kms_Driven  Fuel_Type  Transmission
0    0.666667       0.099913    0.085085        0.5           1.0
1    0.933333       0.103273    0.012813        1.0           1.0
2    0.533333       0.041504    0.009409        1.0           1.0
3    1.000000       0.103056    0.003145        0.5           1.0
4    0.800000       0.084525    0.036629        1.0           1.0
..        ...            ...         ...        ...           ...
220  0.733333       0.070221    0.065103        1.0           1.0
221  0.800000       0.138383    0.119271        0.5           1.0
222  0.800000       0.060468    0.119119        1.0           1.0
223  0.400000       0.115735    0.175043        1.0           1.0
224  0.933333       0.131990    0.017017        0.5           1.0

[225 rows x 5 columns]
     Selling_Price
0         0.133238
1         0.204871
2         0.078797
3         0.262178
4         0.190544
..             ...
220       0.104585
221       0.242120
222    

In [8]:
yTrain=yTrain.iloc[:,0]
yTest=yTest.iloc[:,0]

lenTrain=len(xTrain)
numberInputAttr=len(x.columns)
vector0=np.array([0]*numberInputAttr)
xTrain=np.array(xTrain)

time: 3.8 ms


In [9]:
#batch gradient descent
#print(x)
#print(y)
theta=vector0*1

#train
alpha=0.001
numEpochs=100000
numLoops=0
iLoopGrade=1
while(numLoops<numEpochs):
  errors=np.dot(xTrain, np.transpose(theta))-yTrain
  sumerrors=np.dot(np.transpose(errors), xTrain)
  theta=theta-alpha*sumerrors
  #check convergent
  numLoops+=1
  if numLoops/iLoopGrade>1000:
    iLoopGrade*=1000
    alpha/=10

#result
print('theta: ',theta)

#test
test_results=np.dot(xTest, np.transpose(theta))
test_errors=np.dot(xTest, np.transpose(theta))-yTest
com_res=pd.DataFrame(test_results, columns=['cal_result'])
com_res.insert(1, 'check', yTest, True)
com_res.insert(2, 'error', test_errors, True)
#print(com_res)
print('Evaluate:')
mse=1.0/len(yTest)*np.sum([i*i for i in com_res.error])
mae=1.0/len(yTest)*np.sum([i if i>=0 else -i for i in com_res.error])
print('1-sqrt(mse):', 1-math.sqrt(mse))
print('1-mae:', 1-mae)


theta:  [ 0.17950823  1.23921269 -0.13548296 -0.08302725 -0.01640028]
Evaluate:
1-sqrt(mse): 0.9398541353120583
1-mae: 0.963076920138347
time: 20.3 s


In [10]:
#stochatic gradient descent
theta=vector0*1
lenTrain=len(xTrain)
alpha=0.001
numEpochs=1000000
numLoops=0
iLoopGrade=1
while numLoops<numEpochs:
  error=np.dot(xTrain[numLoops%lenTrain], np.transpose(theta))-yTrain[numLoops%lenTrain]
  total_errors=np.dot(np.transpose(error), xTrain[numLoops%lenTrain])
  theta=theta-alpha*total_errors
  numLoops+=1


#result
print(theta)

#test
test_results=np.dot(xTest, np.transpose(theta))
test_errors=np.dot(xTest, np.transpose(theta))-yTest
com_res=pd.DataFrame(test_results, columns=['cal_result'])
com_res.insert(1, 'check', yTest, True)
com_res.insert(2, 'error', test_errors, True)
#print(com_res)
print('Evaluate:')
mse=1.0/len(yTest)*np.sum([i*i for i in com_res.error])
mae=1.0/len(yTest)*np.sum([i if i>=0 else -i for i in com_res.error])
print('1-sqrt(mse):', 1-math.sqrt(mse))
print('1-mae:', 1-mae)

[ 0.18451797  1.21203263 -0.08373029 -0.08620045 -0.01917089]
Evaluate:
1-sqrt(mse): 0.9405903272561922
1-mae: 0.963476861050256
time: 13.4 s
